In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
import time
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
team_codes_df = pd.read_csv('team_codes.csv')
games_attended_df = pd.read_csv('sports_attendance.csv')

games_attended_df["URL_Date"] = " "
games_attended_df["Year"] = " "

games_attended_df['Year'] = pd.to_datetime(games_attended_df.Date)
games_attended_df['Year'] = games_attended_df['Year'].dt.strftime('%Y')
games_attended_df['URL_Date'] = pd.to_datetime(games_attended_df.Date)
games_attended_df['URL_Date'] = games_attended_df['URL_Date'].dt.strftime('%Y%m%d0')

nba_attendance_df = games_attended_df[games_attended_df.Sport == ('NBA')].copy()
mlb_attendance_df = games_attended_df[games_attended_df.Sport == ('MLB')].copy()
nfl_attendance_df = games_attended_df[games_attended_df.Sport == ('NFL')].copy()

mlb_df = mlb_attendance_df.merge(team_codes_df, how='left', left_on='Home', right_on='MLB_Teams')
mlb_df = mlb_df.drop(columns=['NBA_Teams', 'NBA_Codes', 'NBA_Teams', 'NFL_Teams', 'NFL_Codes', 'NHL_Teams', 'NHL_Codes', 'MLS_Codes'])

mlb_df['URL_Variable'] = mlb_df['MLB_Codes'] + mlb_df['URL_Date']

away_code_df = mlb_attendance_df.merge(team_codes_df, how='left', left_on='Visitor', right_on='MLB_Teams')
away_code_df = away_code_df.drop(columns=['NBA_Teams', 'NBA_Codes', 'NBA_Teams', 'NFL_Teams', 'NFL_Codes', 'NHL_Teams', 'NHL_Codes', 'MLS_Codes'])

mlb_df['Away_Code'] = away_code_df['MLB_Codes']
mlb_df.rename(columns = {'MLB_Codes':'Home_Code'}, inplace = True)
mlb_df


,Date,Home,Visitor,Sport,URL_Date,Year,MLB_Teams,Home_Code,URL_Variable,Away_Code
0,7/27/2007,San Francisco Giants,Florida Marlins,MLB,200707270,2007,San Francisco Giants,SFN,SFN200707270,FLO
1,7/28/2007,San Francisco Giants,Florida Marlins,MLB,200707280,2007,San Francisco Giants,SFN,SFN200707280,FLO
2,9/5/2008,St. Louis Cardinals,Florida Marlins,MLB,200809050,2008,St. Louis Cardinals,SLN,SLN200809050,FLO
3,9/6/2008,St. Louis Cardinals,Florida Marlins,MLB,200809060,2008,St. Louis Cardinals,SLN,SLN200809060,FLO
4,7/25/2009,Los Angeles Angels of Anaheim,Minnesota Twins,MLB,200907250,2009,Los Angeles Angels of Anaheim,ANA,ANA200907250,MIN
5,7/26/2009,Los Angeles Dodgers,Florida Marlins,MLB,200907260,2009,Los Angeles Dodgers,LAN,LAN200907260,FLO
6,8/22/2009,San Diego Padres,St. Louis Cardinals,MLB,200908220,2009,San Diego Padres,SDN,SDN200908220,SLN
7,4/24/2010,San Francisco Giants,St. Louis Cardinals,MLB,201004240,2010,San Francisco Giants,SFN,SFN201004240,SLN
8,9/26/2010,Arizona Diamondbacks,Los Angeles Dodgers,MLB,201009260,2010,Arizona Diamondbacks,ARI,ARI201009260,LAN
9,10/28/2010,San Francisco Giants,Texas Rangers,MLB,201010280,2010,San Francisco Giants,SFN,SFN201010280,TEX


In [3]:
URL_list = mlb_df["URL_Variable"].values.tolist()
Home_code_list = mlb_df["Home_Code"].values.tolist()
Away_code_list = mlb_df["Away_Code"].values.tolist()
Home_team_list = mlb_df["Home"].values.tolist()
Away_team_list = mlb_df["Visitor"].values.tolist()
Year_list = mlb_df["Year"].values.tolist()
Date_list = mlb_df["URL_Date"].values.tolist()
Actual_date_list = mlb_df["Date"].values.tolist()

In [4]:
coded_home_teams = []
coded_away_teams = []

for item in Home_team_list: 
    home_code = re.sub(r'[^a-zA-Z0-9]', '', item)
    coded_home_teams.append(home_code)

for item in Away_team_list: 
    away_code = re.sub(r'[^a-zA-Z0-9]', '', item)
    coded_away_teams.append(away_code)

#  LosAngelesAngelsofAnaheim   
coded_home_teams

['SanFranciscoGiants',
 'SanFranciscoGiants',
 'StLouisCardinals',
 'StLouisCardinals',
 'LosAngelesAngelsofAnaheim',
 'LosAngelesDodgers',
 'SanDiegoPadres',
 'SanFranciscoGiants',
 'ArizonaDiamondbacks',
 'SanFranciscoGiants',
 'SanFranciscoGiants',
 'SanDiegoPadres',
 'SanFranciscoGiants',
 'LosAngelesDodgers',
 'SanFranciscoGiants',
 'LosAngelesDodgers',
 'LosAngelesDodgers',
 'OaklandAthletics',
 'PhiladelphiaPhillies',
 'NewYorkYankees',
 'LosAngelesAngelsofAnaheim',
 'LosAngelesAngelsofAnaheim',
 'LosAngelesAngelsofAnaheim',
 'LosAngelesDodgers',
 'LosAngelesDodgers',
 'LosAngelesDodgers',
 'SanDiegoPadres',
 'SanFranciscoGiants',
 'ColoradoRockies',
 'MilwaukeeBrewers',
 'ChicagoWhiteSox',
 'ChicagoWhiteSox',
 'MinnesotaTwins',
 'ChicagoCubs',
 'StLouisCardinals',
 'CincinnatiReds',
 'MilwaukeeBrewers',
 'DetroitTigers',
 'StLouisCardinals',
 'StLouisCardinals',
 'ChicagoCubs',
 'PittsburghPirates',
 'KansasCityRoyals',
 'ClevelandIndians',
 'TorontoBlueJays',
 'StLouisCardinal

In [5]:
# getting length of list
length = len(URL_list)

dataframe_dictionary = {}

box_score_df_list = []
home_stats_df_list = []
away_stats_df_list = []
game_logistics_df_list = []
pitching_line = []

for i in range(length):
    time.sleep(2)
    
    # Create empty lists to hold table data to be scraped
    away_box_score = []
    home_box_score = []
    box_score = []

    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.baseball-reference.com/boxes/{Home_code_list[i]}/{URL_list[i]}.shtml'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has away team statistics
    away_table_placeholder = soup.select_one(f'#all_{coded_away_teams[i]}batting') 

    # Game box score is commented out in html, so this will grab it out of the comments
    for comment in away_table_placeholder.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            away_table = comment_soup.find("table")

    # Grab data from table and put it into the list created above
    for tr in away_table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        away_box_score.append(tds)

    # Grab the table element that has home team statistics
    home_table_placeholder = soup.select_one(f'#all_{coded_home_teams[i]}batting') 

    # Game box score is commented out in html, so this will grab it out of the comments
    for comment in home_table_placeholder.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            home_table = comment_soup.find("table")

    # Grab data from table and put it into the list created above
    for tr in home_table.select('tr:has(td)'):
        tds2 = [td.get_text(strip=True) for td in tr.select('td')]
        home_box_score.append(tds2)

    # Grab the table element that has game box score
    box_score_table = soup.select('[class*="linescore_wrap"]')

    # Grab data from table and put it into the list created above
    for tr in box_score_table[0].select('tr:has(td)'):
        tds3 = [td.get_text(strip=True) for td in tr.select('td')]
        box_score.append(tds3)

##########################################################################################
    pitching_line.append(box_score[2][0])
##########################################################################################
    
#     # Grab data from table and put it into the list created above
#     for tr in table.select('tr:has(td)'):
#         tds3 = [td.get_text(strip=True) for td in tr.select('td')]
#         box_score.append(tds3)
       
    # Create dataframe for away team statistics
    dataframe_dictionary[Away_code_list[i] + Date_list[i]] = pd.DataFrame(away_box_score)

    # Create an empty list to store away team statistics header information
    away_header_list = []

    # Grab the table header information to use as column headers in our away team statistics dataframe
    for tr in away_table.select('tr:has(th)'):
        ths = [th.get_text(strip=True) for th in tr.select('th')]
        away_header_list.append(ths)

    away_header_list[0].remove("Batting")    
        
    # Create dataframe for home team statistics
    dataframe_dictionary[Home_code_list[i] + Date_list[i]] = pd.DataFrame(home_box_score)

    # Create an empty list to store home team statistics header information
    home_header_list = []

    # Grab the table header information to use as column headers in our home team statistics dataframe
    for tr in home_table.select('tr:has(th)'):
        ths2 = [th.get_text(strip=True) for th in tr.select('th')]
        home_header_list.append(ths2)    
    
    home_header_list[0].remove("Batting")
    
    # Create dataframe for game box score
    dataframe_dictionary['Box' + Date_list[i]] = pd.DataFrame(box_score)

    # Create an empty list to store game box score dataframe header information
    box_score_header_list = []

    # Grab the table header information to use as column headers in our game box score dataframe
    for tr in box_score_table[0].select('tr:has(th)'):
        ths3 = [th.get_text(strip=True) for th in tr.select('th')]
        box_score_header_list.append(ths3)

    box_score_header_list[0][1] = 'Team'
    dataframe_dictionary['Box' + Date_list[i]].columns = box_score_header_list

    # box_score_df.columns = box_score_list
#     final_box_score_df = box_score_df.iloc[: , 1:]
#     final_box_score_df.drop(final_box_score_df.tail(1).index,inplace=True) 

#     home_away = ['Away','Home']
#     final_box_score_df.rename(index={0:'Away'},inplace=True)
#     final_box_score_df.rename(index={1:'Home'},inplace=True)

    
    
#     # Create an list to store game box score dataframe header information
#     box_score_list = ['1Q', '2Q', '3Q', '4Q', 'T']

    # If a game goes into overtime(s), we need to add to the box score list for each overtime period
    Extra_inning_counter = 10
    while len(box_score[0]) > len(box_score_header_list[0]):
        box_score_header_list.insert(-1, f'{Extra_inning_counter}')
        Extra_inning_counter = Extra_inning_counter + 1
        
    # Update the column and row labels
    dataframe_dictionary['Box' + Date_list[i]] = dataframe_dictionary['Box' + Date_list[i]].set_axis(box_score_header_list, axis=1, inplace=False)
#     dataframe_dictionary['Box' + Date_list[i]] = dataframe_dictionary['Box' + Date_list[i]].set_axis([f'{Away_team_list[i]}', f'{Home_team_list[i]}'])
    
    dataframe_dictionary['Box' + Date_list[i]]['Date'] = Actual_date_list[i]
    box_score_df_list.append(dataframe_dictionary['Box' + Date_list[i]])
    
    # Pull the away team player data and combine the lists into one list
    away_starters = away_header_list[1:-1]
#     away_bench = away_header_list[8:-1]
#     away_players = [away_starters,away_bench]

    # Pull the home team player data and combine the lists into one list
    home_starters = home_header_list[1:-1]
#     home_bench = home_header_list[8:-1]
#     home_players = [home_starters,home_bench]

    # Consolodate the away team list of lists into one list
    away_starters_list = [item for sublist in away_starters for item in sublist]
#     away_bench_list = [item for sublist in away_bench for item in sublist]
#     away_roster = away_starters_list + away_bench_list

    # Consolodate the home team list of lists into one list
    home_starters_list = [item for sublist in home_starters for item in sublist]
#     home_bench_list = [item for sublist in home_bench for item in sublist]
#     home_roster = home_starters_list + home_bench_list

    # Add Team Totals to the end of the away team list
    away_starters.append('Team Totals')
    dataframe_dictionary[Away_code_list[i] + Date_list[i]].index = away_starters

    # Add Team Totals to the end of the home team list
    home_starters.append('Team Totals')
    dataframe_dictionary[Home_code_list[i] + Date_list[i]].index = home_starters

    # Set the home and away dataframe column labels
    column_headers = away_header_list[0]
#     column_headers.remove("Starters")
    dataframe_dictionary[Away_code_list[i] + Date_list[i]].columns = column_headers
    dataframe_dictionary[Home_code_list[i] + Date_list[i]].columns = column_headers

    dataframe_dictionary[Away_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    dataframe_dictionary[Home_code_list[i] + Date_list[i]]['Date'] = Actual_date_list[i]
    
    dataframe_dictionary[Away_code_list[i] + Date_list[i]]['Team'] = Away_team_list[i]
    dataframe_dictionary[Home_code_list[i] + Date_list[i]]['Team'] = Home_team_list[i]
    
    home_stats_df_list.append(dataframe_dictionary[Home_code_list[i] + Date_list[i]])
    away_stats_df_list.append(dataframe_dictionary[Away_code_list[i] + Date_list[i]])
    
    # Pull logistical information about the game: attendance, length of game, and venue
    text = soup.get_text()
    attendance_string = re.findall(r"Attendance:\s+\d*,\d*", text)
    
    # A failsafe, as one of the games has no attendance listed causing the loop to break
    if(len(attendance_string) != 0):
        
        # Clean up the attendance string that is pulled 
        new_attendance_string = attendance_string[0].strip('([^a-z]xa)')
        attendance_list = list(new_attendance_string)
        attendance_list.pop(11)
    else:
        attendance_string = "No Attendance Available"
    
    # Clean up the game length string that is pulled 
    game_time_string = re.findall(r"Game Duration:\s+\d:\d*", text)    
    new_game_time_string = game_time_string[0].strip('([^a-z]xa)')
    game_time_list = list(new_game_time_string)
    game_time_list.pop(13)

    # join the lists to create final string
    final_attendance_string = "".join(attendance_list)
    final_game_time_sting = "".join(game_time_list)

    # Create throwaway variables to hold split strings so we can pick out what we want later
    x = final_attendance_string.split(":")
    y = final_game_time_sting.split(" ")
    y[0:2] = [' '.join(y[0:2])]

    # Pick out the strings before the : that we split above
    dataframe_dictionary['Attendance' + Date_list[i]] = pd.DataFrame(x, columns = [x[0]])
    dataframe_dictionary['Attendance' + Date_list[i]] = dataframe_dictionary['Attendance' + Date_list[i]].drop([0])
    dataframe_dictionary['Game Duration' + Date_list[i]] = pd.DataFrame(y, columns = [y[0]])
    dataframe_dictionary['Game Duration' + Date_list[i]] = dataframe_dictionary['Game Duration' + Date_list[i]].drop([0])

    # Join the attendance and game time dataframes into one dataframe
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = pd.concat([dataframe_dictionary['Attendance' + Date_list[i]], dataframe_dictionary['Game Duration' + Date_list[i]]], ignore_index=True, sort=False)
    dataframe_dictionary['Game_Logistics' + Date_list[i]]['Game Duration'] = dataframe_dictionary['Game_Logistics' + Date_list[i]]['Game Duration'].shift(-1)
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = dataframe_dictionary['Game_Logistics' + Date_list[i]].dropna()

    # Scrape the name of the vanue and location and add that to the logistics dataframe
    divparent = soup.find('div', attrs={'class':'scorebox_meta'})
    text = divparent.text
    result = re.split(r'\n', text)
    almost_done = result[1]
    final_result = almost_done.partition("Venue: ")[2]
    real_final_result = final_result.split("Game Duration:")[0]
    dataframe_dictionary['Game_Logistics' + Date_list[i]] = dataframe_dictionary['Game_Logistics' + Date_list[i]].assign(Venue=[real_final_result])

    dataframe_dictionary['Game_Logistics' + Date_list[i]]['Date'] = Actual_date_list[i]
    game_logistics_df_list.append(dataframe_dictionary['Game_Logistics' + Date_list[i]])
    

In [6]:
Away_code_list

['FLO',
 'FLO',
 'FLO',
 'FLO',
 'MIN',
 'FLO',
 'SLN',
 'SLN',
 'LAN',
 'TEX',
 'HOU',
 'SLN',
 'SLN',
 'SLN',
 'BOS',
 'SDN',
 'SLN',
 'TOR',
 'WAS',
 'CIN',
 'LAN',
 'OAK',
 'KCA',
 'SDN',
 'SLN',
 'SFN',
 'SLN',
 'SLN',
 'TBA',
 'CHN',
 'DET',
 'DET',
 'ATL',
 'SEA',
 'ATL',
 'SLN',
 'PIT',
 'KCA',
 'LAN',
 'LAN',
 'SLN',
 'SLN',
 'CHA',
 'NYA',
 'NYA',
 'CHN',
 'ARI',
 'ARI',
 'BAL',
 'WAS',
 'NYN',
 'OAK',
 'TEX',
 'PIT',
 'CLE',
 'DET',
 'OAK',
 'SLN',
 'LAA']

In [7]:
########## GUIDE TO ACCESSING DATAFRAMES ##########

## Pull Home Stats --> dataframe_dictionary['(Home Code)(Date)']

## Pull Away Stats --> dataframe_dictionary['(Away Code)(Date)']

## Pull Box Score --> dataframe_dictionary['Box(Date)']

## Pull Game Logistics --> dataframe_dictionary['Game_Logistics(Date)']

In [8]:
#Test
dataframe_dictionary['Game_Logistics202205070']

,Attendance,Game Duration,Venue,Date
0,"40,113",3:22,Oracle Park,5/7/2022


In [9]:
#Test
dataframe_dictionary['SFN202205070']

,AB,R,H,RBI,BB,SO,PA,BA,OBP,SLG,...,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Team
[Austin SlaterCF],4,1,1,0,1,2,5,.255,.352,.404,...,0.048,-0.043%,0.00%,0.60,-0.3,2,0,,5/7/2022,San Francisco Giants
[Mauricio DubonSS],4,4,2,2,1,0,5,.182,.200,.303,...,0.217,-0.012%,0.12%,0.64,2.1,1,3,HR,5/7/2022,San Francisco Giants
[Brandon Belt1B],1,2,0,0,3,0,4,.239,.386,.463,...,0.101,-0.002%,0.06%,0.70,1.3,6,0,,5/7/2022,San Francisco Giants
[LaMonte Wade Jr.PH-1B],0,1,0,0,1,0,1,.000,.333,.000,...,0.006,0.000%,0.00%,0.17,0.6,1,0,,5/7/2022,San Francisco Giants
[Darin RufDH],4,2,1,2,1,1,5,.190,.296,.240,...,0.114,-0.072%,0.03%,0.75,2.4,,,HR,5/7/2022,San Francisco Giants
[Wilmer Flores3B],3,1,1,6,1,0,5,.269,.340,.419,...,0.259,-0.014%,0.15%,0.84,3.2,0,2,"HR,SF",5/7/2022,San Francisco Giants
[Luis GonzalezLF],4,1,1,1,0,1,5,.318,.365,.432,...,0.013,-0.020%,-0.00%,0.23,0.6,2,0,HBP,5/7/2022,San Francisco Giants
[Thairo Estrada2B],4,0,1,2,0,0,5,.240,.278,.360,...,0.065,-0.008%,0.03%,0.22,0.7,4,2,"2B,SF",5/7/2022,San Francisco Giants
[Joey BartC],4,0,0,0,0,2,5,.164,.313,.345,...,0.003,-0.019%,-0.01%,0.20,-1.5,10,0,"HBP,GDP",5/7/2022,San Francisco Giants
[Mike YastrzemskiRF],4,1,2,0,0,0,4,.288,.354,.356,...,0.033,-0.013%,0.01%,0.34,-0.1,1,0,,5/7/2022,San Francisco Giants


In [10]:
#Test
dataframe_dictionary['SFN202205070']

,AB,R,H,RBI,BB,SO,PA,BA,OBP,SLG,...,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date,Team
[Austin SlaterCF],4,1,1,0,1,2,5,.255,.352,.404,...,0.048,-0.043%,0.00%,0.60,-0.3,2,0,,5/7/2022,San Francisco Giants
[Mauricio DubonSS],4,4,2,2,1,0,5,.182,.200,.303,...,0.217,-0.012%,0.12%,0.64,2.1,1,3,HR,5/7/2022,San Francisco Giants
[Brandon Belt1B],1,2,0,0,3,0,4,.239,.386,.463,...,0.101,-0.002%,0.06%,0.70,1.3,6,0,,5/7/2022,San Francisco Giants
[LaMonte Wade Jr.PH-1B],0,1,0,0,1,0,1,.000,.333,.000,...,0.006,0.000%,0.00%,0.17,0.6,1,0,,5/7/2022,San Francisco Giants
[Darin RufDH],4,2,1,2,1,1,5,.190,.296,.240,...,0.114,-0.072%,0.03%,0.75,2.4,,,HR,5/7/2022,San Francisco Giants
[Wilmer Flores3B],3,1,1,6,1,0,5,.269,.340,.419,...,0.259,-0.014%,0.15%,0.84,3.2,0,2,"HR,SF",5/7/2022,San Francisco Giants
[Luis GonzalezLF],4,1,1,1,0,1,5,.318,.365,.432,...,0.013,-0.020%,-0.00%,0.23,0.6,2,0,HBP,5/7/2022,San Francisco Giants
[Thairo Estrada2B],4,0,1,2,0,0,5,.240,.278,.360,...,0.065,-0.008%,0.03%,0.22,0.7,4,2,"2B,SF",5/7/2022,San Francisco Giants
[Joey BartC],4,0,0,0,0,2,5,.164,.313,.345,...,0.003,-0.019%,-0.01%,0.20,-1.5,10,0,"HBP,GDP",5/7/2022,San Francisco Giants
[Mike YastrzemskiRF],4,1,2,0,0,0,4,.288,.354,.356,...,0.033,-0.013%,0.01%,0.34,-0.1,1,0,,5/7/2022,San Francisco Giants


In [11]:
#Test
dataframe_dictionary['Box202205070']

,,Team,1,2,3,4,5,6,7,8,9,R,H,E,Date
0,via Sports Logos.netAbout logos,St. Louis Cardinals,1,2,0,1,0,0,3,0,0,7,14,3,5/7/2022
1,via Sports Logos.netAbout logos,San Francisco Giants,4,4,0,1,0,0,1,3,X,13,9,0,5/7/2022
2,WP: Logan Webb (4-1) • LP: Steven Matz (3-2),None,None,None,None,None,None,None,None,None,None,None,None,None,5/7/2022


In [12]:
total_box_score_df = pd.concat(box_score_df_list)

total_box_score_df

,,1,10,11,12,13,14,15,2,3,...,5,6,7,8,9,Date,E,H,R,Team
0,via Sports Logos.netAbout logos,1,NaN,NaN,NaN,NaN,NaN,NaN,0,4,...,0,1,1,0,2,7/27/2007,2,15,10,Florida Marlins
1,via Sports Logos.netAbout logos,2,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,4,6,0,0,X,7/27/2007,0,14,12,San Francisco Giants
2,WP: Vinnie Chulk (4-2) • LP: Renyel Pinto (2-4),None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,...,None,None,None,None,None,7/27/2007,None,None,None,None
0,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,3,...,0,0,0,0,0,7/28/2007,2,6,3,Florida Marlins
1,via Sports Logos.netAbout logos,1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,1,0,0,2,7/28/2007,0,9,4,San Francisco Giants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,via Sports Logos.netAbout logos,4,NaN,NaN,NaN,NaN,NaN,NaN,4,0,...,0,0,1,3,X,5/7/2022,0,9,13,San Francisco Giants
2,WP: Logan Webb (4-1) • LP: Steven Matz (3-2),None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,...,None,None,None,None,None,5/7/2022,None,None,None,None
0,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,1,0,0,0,0,3/30/2023,0,5,1,Los Angeles Angels
1,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,2,X,3/30/2023,1,6,2,Oakland Athletics


In [13]:
total_home_stats_df = pd.concat(home_stats_df_list)

team_col = total_home_stats_df.pop("Team")
total_home_stats_df.insert(0, "Team", team_col)

total_home_stats_df

,Team,AB,R,H,RBI,BB,SO,PA,BA,OBP,...,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date
[Dave RobertsCF],San Francisco Giants,4,1,2,0,1,1,5,.262,.327,...,1.00,0.097,-0.095,0%,0.05,0.2,3,0,SB,7/27/2007
[Omar VizquelSS],San Francisco Giants,4,1,1,1,0,0,5,.246,.295,...,1.36,0.172,-0.104,0.00%,0.06,-0.4,2,2,SH,7/27/2007
[Ray Durham2B],San Francisco Giants,4,1,2,3,0,1,5,.240,.307,...,0.96,0.272,-0.072,0.01%,0.04,1.2,1,1,"HR,2B,SF",7/27/2007
[Randy MessengerP],San Francisco Giants,0,0,0,0,0,0,0,.000,.000,...,,,,,,,0,0,,7/27/2007
[Barry BondsLF],San Francisco Giants,1,2,1,1,4,0,5,.281,.499,...,0.36,0.154,0.000,0.00%,0.02,2.1,2,0,HR,7/27/2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[Zach JacksonP],Oakland Athletics,,,,,,,,,,...,,,,,,,0,0,,3/30/2023
[Domingo AcevedoP],Oakland Athletics,,,,,,,,,,...,,,,,,,0,0,,3/30/2023
[Trevor MayP],Oakland Athletics,,,,,,,,,,...,,,,,,,0,0,,3/30/2023
[Dany JiménezP],Oakland Athletics,,,,,,,,,,...,,,,,,,0,0,,3/30/2023


In [14]:
total_away_stats_df = pd.concat(away_stats_df_list)

team_col = total_away_stats_df.pop("Team")
total_away_stats_df.insert(0, "Team", team_col)

total_away_stats_df

,Team,AB,R,H,RBI,BB,SO,PA,BA,OBP,...,aLI,WPA+,WPA-,cWPA,acLI,RE24,PO,A,Details,Date
[Hanley RamirezSS],Florida Marlins,5,1,3,4,0,0,6,.339,.391,...,1.01,0.245,-0.071,0.03%,0.26,1.7,4,2,"HR,2B,SF",7/27/2007
[Dan Uggla2B],Florida Marlins,6,1,1,0,0,0,6,.257,.331,...,0.93,0.039,-0.127,-0.01%,0.24,-1.0,1,3,2B,7/27/2007
[Miguel Cabrera3B],Florida Marlins,4,1,2,1,2,0,6,.332,.405,...,0.78,0.125,-0.051,0.01%,0.20,0.9,0,4,IW,7/27/2007
[Josh WillinghamLF],Florida Marlins,2,1,1,0,3,0,5,.263,.375,...,0.82,0.091,-0.046,0.01%,0.21,0.8,2,0,GDP,7/27/2007
[Mike Jacobs1B],Florida Marlins,5,1,2,2,0,1,5,.277,.328,...,1.35,0.250,-0.088,0.03%,0.35,0.7,8,1,2B,7/27/2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
[Shohei OhtaniP],Los Angeles Angels,,,,,,,,,,...,,,,,,,0,0,,3/30/2023
[Jimmy HergetP],Los Angeles Angels,,,,,,,,,,...,,,,,,,0,1,,3/30/2023
[Aaron LoupP],Los Angeles Angels,,,,,,,,,,...,,,,,,,0,0,,3/30/2023
[Ryan TeperaP],Los Angeles Angels,,,,,,,,,,...,,,,,,,0,0,,3/30/2023


In [15]:
total_game_logistics_df = pd.concat(game_logistics_df_list)
total_game_logistics_df

,Attendance,Game Duration,Venue,Date
0,"42,831",3:15,AT&T Park,7/27/2007
0,"43,001",2:55,AT&T Park,7/28/2007
0,"42,633",3:13,Busch Stadium III,9/5/2008
0,"42,814",2:24,Busch Stadium III,9/6/2008
0,"35,922",3:09,Angel Stadium of Anaheim,7/25/2009
0,"48,597",3:15,Dodger Stadium,7/26/2009
0,"38,156",2:34,Petco Park,8/22/2009
0,"41,785",2:26,AT&T Park,4/24/2010
0,"37,911",2:59,Chase Field,9/26/2010
0,"43,622",3:17,AT&T Park,10/28/2010


In [17]:
index_list = total_box_score_df.index.tolist()
home_away_list = []


for i in range(len(index_list)):
    if (i % 2) == 0:
        home_away_list.append('Away')
    else:
        home_away_list.append('Home')


total_box_score_df = total_box_score_df.set_index('Date')
# total_box_score_df.insert(0, 'Team', index_list)
total_box_score_df.insert(0, 'Home_Away', home_away_list)
total_box_score_df

,Home_Away,,1,10,11,12,13,14,15,2,...,4,5,6,7,8,9,E,H,R,Team
Date,,,,,,,,,,,,,,,,,,,,,
"(7/27/2007,)",Away,via Sports Logos.netAbout logos,1,NaN,NaN,NaN,NaN,NaN,NaN,0,...,1,0,1,1,0,2,2,15,10,Florida Marlins
"(7/27/2007,)",Home,via Sports Logos.netAbout logos,2,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,4,6,0,0,X,0,14,12,San Francisco Giants
"(7/27/2007,)",Away,WP: Vinnie Chulk (4-2) • LP: Renyel Pinto (2-4),None,NaN,NaN,NaN,NaN,NaN,NaN,None,...,None,None,None,None,None,None,None,None,None,None
"(7/28/2007,)",Home,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,0,0,0,0,2,6,3,Florida Marlins
"(7/28/2007,)",Away,via Sports Logos.netAbout logos,1,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,0,1,0,0,2,0,9,4,San Francisco Giants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(5/7/2022,)",Away,via Sports Logos.netAbout logos,4,NaN,NaN,NaN,NaN,NaN,NaN,4,...,1,0,0,1,3,X,0,9,13,San Francisco Giants
"(5/7/2022,)",Home,WP: Logan Webb (4-1) • LP: Steven Matz (3-2),None,NaN,NaN,NaN,NaN,NaN,NaN,None,...,None,None,None,None,None,None,None,None,None,None
"(3/30/2023,)",Away,via Sports Logos.netAbout logos,0,NaN,NaN,NaN,NaN,NaN,NaN,0,...,0,1,0,0,0,0,0,5,1,Los Angeles Angels


In [18]:
pitching_line_df = pd.DataFrame(pitching_line)
pitching_line_df

,0
0,WP: Vinnie Chulk (4-2) • LP: Renyel Pinto (2-4)
1,WP: Jack Taschner (2-0) • LP: Kevin Gregg (0-4)
2,WP: Arthur Rhodes (4-1) • LP: Ryan Franklin (5...
3,WP: Todd Wellemeyer (12-6) • LP: Scott Olsen (...
4,WP: Matt Palmer (8-1) • LP: Nick Blackburn (8-5)
5,WP: Chris Volstad (8-9) • LP: Jason Schmidt (1-1)
6,WP: Chris Carpenter (14-3) • LP: Kevin Correia...
7,WP: Barry Zito (3-0) • LP: Adam Wainwright (3-...
8,WP: Sam Demel (2-1) • LP: Jonathan Broxton (5-...
9,WP: Matt Cain (2-0) • LP: C.J. Wilson (1-2)


In [19]:
total_game_logistics_df.to_csv('total_game_logistics.csv')
total_away_stats_df.to_csv('total_away_stats.csv')
total_home_stats_df.to_csv('total_home_stats.csv')
total_box_score_df.to_csv('total_box_score_df.csv')
pitching_line_df.to_csv('pitching_line_df.csv')
# hof_df.to_csv('mlb_hof.csv')
# mvp_df.to_csv('mlb_mvp.csv')
# all_league_df.to_csv('all_league_players.csv')
# allstar_df.to_csv('allstars.csv')
# agg_dpoy_seen_df.to_csv('dpoy.csv')

In [ ]:
box_score_header_list